# How to run a simple variational algorithm with roqoqo

This example notebook is designed to show how to run a very simple variational algorithm with the rust core of qoqo - roqoqo. The variational algorithm will be a very simple example of a Variational Hamiltonian Ansatz (VHA), the code does not aim to get the best result possible but to show a very simple example.

For detailed discussions of variational algorithms, VHA and different variants of these algorithms see the literature (e.g. http://arxiv.org/abs/1304.3061, http://arxiv.org/abs/1509.04279).


## Example: VHA for spins using roqoqo

The goal of a variational algorithm is simple: finding a good approximation of the ground state of a physical system defined by a Hamiltonian H by minimizing the expectation value of H with respect to a set of trial states $| \psi (\vec{\theta} )>$.
The optimization is carried out by optimizing the classical parameters $\vec{\theta}$ defining the trial states.  
By definition the ground state is the state with the lowest energy, so this approach will find the ground state if the ground-state is in the set of possible trial states and the classical optimizer is successfull.  

The trial states are prepared by applying a set of unitary transformations to an initial state
$$
| \psi (\vec{\theta}) > = \prod_j U_j (\vec{\theta}) | \psi_{\textrm{init}} >.
$$
In a VHA the ansatz is to assume that the Hamiltonian can be separated into partial Hamiltonians
$$
H = \sum_{\alpha} H_{\alpha}
$$
and use the time evolution under these partial Hamiltonians as the ansatz for the unitary transformations
$$
| \psi (\vec{\theta}) > = \prod_k^{N} \prod_{\alpha} \exp(-i \theta_{k,\alpha} H_{\alpha}) | \psi_{\textrm{init}}>,
$$
where N is the number of iterations of the pseudo time evolution and $(\theta_{k,\alpha})$ is the variational pseudo time.

Here we use as a sample Hamiltonian a one-dimensional spin chain with three sites
$$
H = H_0 + H_1 + H_2
$$
where $H_0$ is the magnetic onsite energy
$$
H_0 = B \left(\sigma^z_0 + \sigma^z_1 + \sigma^z_0\right),
$$
$H_1$ is the hopping between even and odd sites
$$
H_1 = t \sigma^x_0\sigma^x_1,
$$
and $H_2$ is the hopping between odd and even sites
$$
H_2 = t \sigma^x_1\sigma^x_2.
$$


### The VHA example consists of the following steps:

1. Prepare helper functions to define a time evolution circuit that contains the variational parametere $\vec{\theta}$ as symbolic parameters.
2. Define a circuit to initialize the initial state.
3. Define basis rotation (BR) measurement circuits and the measurement information containing t and B to measure the expectation value of $H$. 
4. Combine the parts in a qoqo quantum program that can be called with the free parameters and directly return the expectation values.
5. Use the compact quantum program to optimize the free parameters $\vec{\theta}$.

Additionally, an exact solution of the Hamiltonian is presented at the end to compare the exact results with the calculated solution.

In [3]:
:dep num-complex = "0.4"
:dep roqoqo = "0.9.0"
:dep roqoqo-quest = "0.2.0"
:dep qoqo_calculator = "0.5"
//:dep cobyla = "0.1.2" // compilation fails

In [4]:
extern crate ndarray;
extern crate num_complex;
extern crate roqoqo;
extern crate roqoqo_quest;
// extern crate cobyla;

use roqoqo::{Circuit, operations::*, registers::*, QuantumProgram};
// for measuring observables
use roqoqo::measurements::{BasisRotationInput, BasisRotation};
// simulation and measurement of the circuit is handled by the QuEST interface
use roqoqo_quest::Backend;
use qoqo_calculator::CalculatorFloat;
use std::collections::HashMap;
use std::convert::TryInto;
use std::vec::*;

use ndarray::{array, Array, Array1, ArrayView1};
use num_complex::Complex64;

### Unitary time evolution

We construct circuits that apply time evolution under the even and odd hopping Hamiltonians and under the magnetic field using variables t (hopping_parameter) and B (magnetic_field).  
For each iteration of the evolution we get free symbolic parameters theta_even_i, theta_odd_i and theta_z_i. 

In [5]:
/// Helper function to return a circuit for the time-like evolution under even-to-odd hopping.
///
/// # Arguments
/// * thetasymb: symbolic parameter 'theta' of the even-to-odd time-like evolution.
/// * number_qubits: number of qubits in the system.
/// * hopping_parameter: parameter 't' for the hopping term of the Hamiltonian.
pub fn create_even_hopping_circuit(thetasymb: CalculatorFloat, number_qubits: usize, hopping_parameter: f64) -> Circuit {
    let mut circuit = Circuit::new();

    // Representation of the \sigma^x\sigma^x interaction between two spins by the CNOT and Rotation gates
    for k in (0..number_qubits-1).step_by(2) {
        // Basis rotation into z-basis
        circuit.add_operation(Hadamard::new(k));
        circuit.add_operation(Hadamard::new(k+1));
        // Interaction
        circuit.add_operation(CNOT::new(k+1, k));
        circuit.add_operation(RotateZ::new(k, (thetasymb.clone() * hopping_parameter).to_string().into()));
        circuit.add_operation(CNOT::new(k+1, k));
        // Basis rotation back into x-basis
        circuit.add_operation(Hadamard::new(k));
        circuit.add_operation(Hadamard::new(k+1))
    }

    return circuit;
}

/// Helper function to return a circuit for the time-like evolution under odd-to-even hopping.
///
/// # Arguments
/// * thetasymb: symbolic parameter 'theta' of the odd-to-even time-like evolution.
/// * number_qubits: number of qubits in the system.
/// * hopping_parameter: parameter 't' for the hopping term of the Hamiltonian.
pub fn create_odd_hopping_circuit(thetasymb: CalculatorFloat, number_qubits: usize, hopping_parameter: f64) -> Circuit {
    let mut circuit = Circuit::new();

    // Representation of the \sigma^x\sigma^x interaction between two spins by the CNOT and Rotation gates
    for k in (1..number_qubits-1).step_by(2) {
        // Basis rotation into z-basis
        circuit.add_operation(Hadamard::new(k));
        circuit.add_operation(Hadamard::new(k+1));
        // Interaction
        circuit.add_operation(CNOT::new(k+1, k));
        circuit.add_operation(RotateZ::new(k, (thetasymb.clone() * hopping_parameter).to_string().into()));
        circuit.add_operation(CNOT::new(k+1, k));
        // Basis rotation back into x-basis
        circuit.add_operation(Hadamard::new(k));
        circuit.add_operation(Hadamard::new(k+1))
    }

    // Periodic boundary conditions
    circuit.add_operation(Hadamard::new(number_qubits-1));
    circuit.add_operation(Hadamard::new(0));
    circuit.add_operation(CNOT::new(0, number_qubits - 1));
    circuit.add_operation(RotateZ::new(number_qubits - 1, (thetasymb.clone() * hopping_parameter).to_string().into()));
    circuit.add_operation(CNOT::new(0, number_qubits - 1));
    circuit.add_operation(Hadamard::new(number_qubits-1));
    circuit.add_operation(Hadamard::new(0));

    return circuit;
}

/// Helper function to return a circuit for the time-like evolution under magnetic field.
///
/// # Arguments
/// * thetasymb: symbolic parameter 'theta' of the z-rotation.
/// * number_qubits: number of qubits in the system
/// * magnetic_field: parameter 'B' for the magnetic term of the Hamiltonian.
pub fn create_magnetic_field_circuit(thetasymb: CalculatorFloat, number_qubits: usize, magnetic_field: f64) -> Circuit {
    let mut circuit = Circuit::new();

    // Representation of the \sigma^x\sigma^x interaction between two spins by the CNOT and Rotation gates
    for k in 0..number_qubits {
        circuit.add_operation(RotateZ::new(k, (thetasymb.clone() * magnetic_field).to_string().into()));
    }

    return circuit;
}

In [6]:
/// Function to construct a Circuit for the unitary evolution.
///
/// Arguments
/// * iter_evolution: number of iterations of evolution, minimum 1.
/// * number_qubits: number of qubits in the system
/// * hopping_parameter: parameter 't' for the hopping term of the Hamiltonian.
/// * magnetic_field: parameter 'B' for the magnetic term of the Hamiltonian.
///
/// # Returns
/// Circuit for the unitary evolution
pub fn create_evolution_circuit(
    iter_evolution: usize,
    number_qubits: usize,
    hopping_parameter: f64,
    magnetic_field: f64,
) -> Circuit {
    let mut circuit = Circuit::new();

    // here: theta_even_i, theta_odd_i and theta_z_i are symbolic parameters (free variational parameters)
    for i in 0..iter_evolution {
        circuit += create_even_hopping_circuit(
            ("theta_even_".to_string() + &i.to_string()).into(),
            number_qubits,
            hopping_parameter,
        );
        circuit += create_odd_hopping_circuit(
            ("theta_odd_".to_string() + &i.to_string()).into(),
            number_qubits,
            hopping_parameter,
        );
        circuit += create_magnetic_field_circuit(
            ("theta_z_".to_string() + &i.to_string()).into(),
            number_qubits,
            magnetic_field,
        );
    }

    return circuit;
}

### Initialization of the state vector by roqoqo

In principle the initial state $\left|\psi_{\textrm{init}}\right>$ has to be prepared with a full quantum circuit (e.g. http://arxiv.org/abs/1711.05395), since we want to keep the example simple we will use a "cheated" initial state instead.

When using a simulator backend one can use a PRAGMA operation to "cheat" and directly set the state vector on the simulator. 
The n-th entry in the state vector corresponds to the basis state $|b(n,2) b(n,1) b(b,0)>$ where $b(n,k)$ gives the k-th enty of the binary representation of n.
$$
0 \leftrightarrow |000>
$$
$$
1 \leftrightarrow |001>
$$
$$
2 \leftrightarrow |010>
$$
and so on.

We choose a starting vector that is 50% in the single excitation subspace and 50% fully occupied 
$$
|\psi_{\textrm{init}}> =  \frac{1}{\sqrt{6}}|001> + \frac{1}{\sqrt{6}} |010> + \frac{1}{\sqrt{6}} |100> + \frac{1}{\sqrt{2}} |111>.
$$
We do not include extra terms to change the number of excitations in the VHA ansatz. Choosing a good initial guess for the number of excitations helps with convergence. For VHA variations that automatically derive the right number of excitations see for example https://doi.org/10.1088/2058-9565/abe568.

### Basis rotation (BR) measurement to get the expectation values

We construct the basis rotation circuits for the measurement of the separate parts of the Hamiltonian.
The magnetic field part of the Hamiltonian contains only $\sigma^z$ operators and can be measured in the z-basis of all qubits. The hopping parts of the Hamiltonian contain only products of $\sigma^x$ operators and can be measured in the x-basis of all qubits. For more information on the basis rotation measurement see the "Introduction to qoqo" example.  

After constructing the measurement circuit and the measurement information we combine everything into one qoqo measurement.

### Combining the parts to QuantumProgram

To execute the optimization we combine all the constructed circuits and put them in one qoqo quantum programm. With QuantumProgram we only need to provide the free parameters and get back the measured expectation values. 

Finally, all the previous steps are collected in a wrapper_function which takes free parameters as argument and returns a single value as result. The wrapper_function can be used to run the optimization routine.

In [10]:
fn wrapper_function(theta: &Vec<f64>) -> f64 {
    // Seting up initial state vector
    let initial_statevector: Array1<Complex64> = array![
        Complex64::new(0.0, 0.0),
        Complex64::new(1.0/6_f64.sqrt(), 0.0),
        Complex64::new(1.0/6_f64.sqrt(), 0.0),
        Complex64::new(0.0, 0.0),
        Complex64::new(1.0/6_f64.sqrt(), 0.0),
        Complex64::new(0.0, 0.0),
        Complex64::new(0.0, 0.0),
        Complex64::new(1.0/2_f64.sqrt(), 0.0),
    ];

    let mut circuit_init = Circuit::new();

    circuit_init += PragmaSetStateVector::new(initial_statevector.clone());

    println!("Step 1: Initialization circuit constructed.");

    // variables and parameters
    let number_qubits: usize = 3;
    let magnetic_field: f64 = 1.0;
    let hopping_parameter: f64 = 3.0;
    // In order to achieve better minimization results we default to several iterations of (pseudo) time-evolution
    let iter_evolution: usize = 4;
    // Construction the evolution circuit
    let circuit_evolution: Circuit = create_evolution_circuit(iter_evolution, number_qubits, hopping_parameter, magnetic_field);

    println!("Step 2: Constructed evolution circuit.");


    // Setting up two basis rotation measurement circuits since we need to measure in two different bases.
    let mut circuit_x_basis_measurement = Circuit::new();
    let mut circuit_z_basis_measurement = Circuit::new();

    // Adding 'DefinitionBit' to the circuit defines the classical bit registers used to store the qubit readout.
    circuit_x_basis_measurement += DefinitionBit::new("ro_x".to_string(), number_qubits, true);
    circuit_z_basis_measurement += DefinitionBit::new("ro_z".to_string(), number_qubits, true);

    // Basis rotation with the Hadamard gate: Bring all qubits into z-basis.
    for i in 0..number_qubits {
        circuit_x_basis_measurement.add_operation(Hadamard::new(i));
    };

    // Add measurement operation to all circuits to write the measured values into the classical registers.
    let number_measurements: usize = 10000;
    circuit_z_basis_measurement.add_operation(PragmaRepeatedMeasurement::new("ro_z".to_string(), number_measurements, None));
    circuit_x_basis_measurement.add_operation(PragmaRepeatedMeasurement::new("ro_x".to_string(), number_measurements, None));

    // Setting up measurement input determining which expectation values of PauliProducts are measured from which circuit
    // and how they are combined linearly
    let mut measurement_input = BasisRotationInput::new(number_qubits, false);
    measurement_input.add_pauli_product("ro_z".to_string(), vec![0]);
    measurement_input.add_pauli_product("ro_z".to_string(), vec![1]);
    measurement_input.add_pauli_product("ro_z".to_string(), vec![2]);
    measurement_input.add_pauli_product("ro_x".to_string(), vec![0,1]);
    measurement_input.add_pauli_product("ro_x".to_string(), vec![1,2]);
    measurement_input.add_pauli_product("ro_x".to_string(), vec![0,2]);
    // Defining and adding the linear combinations of Pauli products that give the expectation values
    let linear_exp_val = HashMap::from([
        (0, magnetic_field),
        (1, magnetic_field),
        (2, magnetic_field),
        (3, hopping_parameter),
        (4, hopping_parameter),
        (5, hopping_parameter),
    ]);
    measurement_input.add_linear_exp_val("energy".to_string(), linear_exp_val);

    println!("Step 3: Measurement circuits constructed. Measurement input prepared.");
    println!("{}{:?}", "In z-basis: ", circuit_z_basis_measurement);
    println!("{}{:?}", "In x-basis: ", circuit_x_basis_measurement);

    // Construct basis rotation measurement to get the expectation values.
    let measurement = BasisRotation {
        input: measurement_input,
        circuits: vec![
            circuit_init.clone() + circuit_evolution.clone() + circuit_z_basis_measurement.clone(),
            circuit_init.clone() + circuit_evolution.clone() + circuit_x_basis_measurement.clone(),
        ],
        constant_circuit: None,
    };

    // One needs to define a backend where the program is executed.
    let backend = Backend::new(number_qubits);
    // QuantumProgram takes the prepared list of circuits and the list of free parameter names (the symbolic values in the circuit).
    let program = QuantumProgram::BasisRotation {
        measurement: measurement,
        input_parameter_names: vec![
            "theta_even_0".to_string(),"theta_odd_0".to_string(), "theta_z_0".to_string(),
            "theta_even_1".to_string(),"theta_odd_1".to_string(), "theta_z_1".to_string(),
            "theta_even_2".to_string(),"theta_odd_2".to_string(), "theta_z_2".to_string(),
            "theta_even_3".to_string(),"theta_odd_3".to_string(), "theta_z_3".to_string(),
        ],
    };

    println!("Step 4: QuantumProgram constructed.");

    let result = program.run(backend, theta).unwrap().unwrap()["energy"];
    return result;
}


Error: mismatched types

### 5. Optimization of free parameters
Minimiization routine to optimize the free parameters. The wrapper_function consisting of the quantum program has 12 free parameters: theta_even, theta_odd and theta_z for each of the 4 iterations of evolution.

In [8]:
let theta = vec![0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];
println!("{:?}", wrapper_function(&theta));

Step 1: Initialization circuit constructed.
Step 2: Constructed evolution circuit.
Step 3: Measurement circuits constructed. Measurement input prepared.
In z-basis: Circuit { definitions: [DefinitionBit(DefinitionBit { name: "ro_z", length: 3, is_output: true })], operations: [PragmaRepeatedMeasurement(PragmaRepeatedMeasurement { readout: "ro_z", number_measurements: 10000, qubit_mapping: None })] }
In x-basis: Circuit { definitions: [DefinitionBit(DefinitionBit { name: "ro_x", length: 3, is_output: true })], operations: [Hadamard(Hadamard { qubit: 0 }), Hadamard(Hadamard { qubit: 1 }), Hadamard(Hadamard { qubit: 2 }), PragmaRepeatedMeasurement(PragmaRepeatedMeasurement { readout: "ro_x", number_measurements: 10000, qubit_mapping: None })] }
Step 4: QuantumProgram constructed.
7.1815999999999995


In [9]:
// // DOES NOT WORK
// // gives an error message on mismatched types for Array
// :dep optimize = "0.1.0"
// extern crate optimize;
// use optimize::{NelderMead, NelderMeadBuilder, Minimizer};

// let minimizer = NelderMeadBuilder::default()
//     .xtol(1e-6f64)
//     .ftol(1e-6f64)
//     .maxiter(50000)
//     .build()
//     .unwrap();

// let theta = vec![0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];
// let args = Array::from_vec(theta);

// let ans = minimizer.minimize(&wrapper_function, args.view());
// println!("Final optimized arguments: {}", ans);


Error: mismatched types

In [13]:
// // DOES NOT WORK
// :dep optimization="0.2.0"
// extern crate optimization;

// use optimization::{Minimizer, GradientDescent};

// // we use a simple gradient descent scheme
// let minimizer = GradientDescent::new();

// // perform the actual minimization
// let solution = minimizer.minimize(&wrapper_function, theta);

// println!("Found solution for Rosenbrock function at f({:?}) = {:?}",
//     solution.position, solution.value);

// // This optimizer gives an error message "the trait "Summation1" is not implemented for f64"

Error: the module `types` is defined here

Error: the trait bound `for<'r> fn(&'r std::vec::Vec<f64>) -> f64 {wrapper_function}: Summation1` is not satisfied

In [ ]:
// println!("{}{:?}{}", , "==> Optimized parameters theta: ", final_result.x, ".);
// println!("{}{:?}{}", "==> Calculated approximate Energy value: ", final_result.fun, ".");

## PART II: Compare the calculated (approximate) result to the exact classical solution

Here we present an exact solution of the sample Hamiltonian to compare the exact results to the calculated solution of the VHA method.

In [6]:
:dep num-complex = "0.4"
extern crate eigenvalues;
extern crate nalgebra;
extern crate ndarray;
extern crate num_complex;

use nalgebra as na;
use na::DMatrix;
use na::ComplexField;
use ndarray::{array, Array2};
use num_complex::Complex64;
use std::vec::*;

In [7]:
// First, construct Hamiltonian
// Pauli matrices
let sigmax_array: Array2<Complex64> = array![
    [Complex64::new(0.0, 0.0), Complex64::new(1.0, 0.0)],
    [Complex64::new(1.0, 0.0), Complex64::new(0.0, 0.0)],
];
let sigmay_array: Array2<Complex64> = array![
    [Complex64::new(0.0, 0.0), Complex64::new(0.0, 1.0)],
    [Complex64::new(0.0, -1.0), Complex64::new(0.0, 0.0)],
];
let sigmaz_array: Array2<Complex64> = array![
    [Complex64::new(1.0, 0.0), Complex64::new(0.0, 0.0)],
    [Complex64::new(0.0, 0.0), Complex64::new(-1.0, 0.0)],
];
// Identity
let identity_array: Array2<Complex64> = array![
    [Complex64::new(1.0, 0.0), Complex64::new(0.0, 0.0)],
    [Complex64::new(0.0, 0.0), Complex64::new(1.0, 0.0)],
];

// convert 2d array to nalgebra::DMatrix to use the kronecker product function
/// Helper function to convert a two-dimensional ndarray to a matrix
fn convert_matrix(customarray: Array2<Complex64>) -> na::DMatrix<Complex64> {
    let dim = customarray.dim();
    na::DMatrix::<Complex64>::from_iterator(dim.0, dim.1, customarray.t().iter().cloned())
}

let sigmax = convert_matrix(sigmax_array);
let sigmay = convert_matrix(sigmay_array);
let sigmaz = convert_matrix(sigmaz_array);
let identity = convert_matrix(identity_array);

// hopping term for 3 qubits
let hopping_factor = Complex64::new(3.0, 0.0);
let H_hopping = (
    (sigmax.clone().kronecker(&sigmax)).kronecker(&identity)
    + (identity.clone().kronecker(&sigmax)).kronecker(&sigmax)
    + (sigmax.clone().kronecker(&identity)).kronecker(&sigmax)
) * hopping_factor;

// magnetic term for 3 qubits
let magnetic_factor = Complex64::new(1.0, 0.0);
let H_magnetic = (
    (sigmaz.clone().kronecker(&identity)).kronecker(&identity)
    + (identity.clone().kronecker(&sigmaz)).kronecker(&identity)
    + (identity.clone().kronecker(&identity)).kronecker(&sigmaz)
) * magnetic_factor;

// total Hamiltonian
let H_matrix: na::DMatrix<Complex64> = H_hopping + H_magnetic;

// get the exact eigenvalues
let mut eigenvalues = vec![];
for k in &H_matrix.clone().eigenvalues().unwrap() {
    eigenvalues.push(k.clone().real());
}
eigenvalues.sort_by(|a, b| a.partial_cmp(b).unwrap());
println!("{} {:?}", ">> Calculated exact eigenvalues: ", eigenvalues);

// final print-out
// let delta = final_result.fun - eigenvalues[0];
// println!("{}{}", ">> Difference between VHA result and exact result: ", delta);

>> Calculated exact eigenvalues:  [-5.211102550927977, -4.000000000000003, -3.999999999999991, -2.0000000000000004, -1.9999999999999982, -1.291502622129181, 9.211102550928004, 9.291502622129183]
